In [ ]:
# Sports Query Notebook
# This notebook allows you to query recent sports results

# !pip install langchain langchain-openai langchain-community langchain-core tavily-python

import os
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.chains import create_structured_output_chain
from typing import Dict, List, Optional, Any
import json

# Set your API keys
# You need to set these environment variables or replace with your actual keys
os.environ["OPENAI_API_KEY"] = "your-openai-api-key"  # Replace with your OpenAI API key
os.environ["TAVILY_API_KEY"] = "your-tavily-api-key"  # Replace with your Tavily API key

# Define the search tool
search_tool = TavilySearchResults(max_results=5)

# Define our large language model
llm = ChatOpenAI(model="gpt-4", temperature=0)  # You can replace with your preferred model


In [ ]:
# Create a prompt template for retrieving sports results
prompt_template = """
You are a helpful sports results assistant. Given the following query, use the search results to provide 
a concise and accurate answer about the sports match. If the information is not found in the search results, 
state that clearly.

Query: {query}

Search Results: {search_results}

Answer the query based on the search results. Be specific about the score, the date of the match, any 
important highlights, and which competition/tournament the match was part of. If the match hasn't 
happened yet, provide information about when it's scheduled.

Response:
"""

prompt = ChatPromptTemplate.from_template(prompt_template)

# Create a function to perform the search and get the results
def search_sports_result(query: str) -> List[Dict]:
    """
    Search for sports results using Tavily search.
    """
    results = search_tool.invoke(query + " recent result score")
    return results

# Create the chain to get a structured response
def get_sports_result(query: str) -> str:
    """
    Get a structured response for a sports query.
    
    Args:
        query: A string query about sports results, e.g., "What was the result of yesterday's game between Arsenal and PSG?"
        
    Returns:
        A string with a structured response containing the match result and relevant information.
    """
    # Perform the search
    search_results = search_sports_result(query)
    
    # Create chain
    chain = prompt | llm | StrOutputParser()
    
    # Run the chain
    result = chain.invoke({"query": query, "search_results": search_results})
    
    return result

# Example usage
if __name__ == "__main__":
    # You can test with different queries
    query = "What was the result of yesterday's game between Arsenal and PSG?"
    result = get_sports_result(query)
    print(result)

# Interactive usage with a simple UI using IPython widgets
try:
    from IPython.display import display, HTML
    import ipywidgets as widgets
    
    def handle_submit(btn):
        query = text_input.value
        with output:
            output.clear_output()
            print(f"Searching for: {query}")
            result = get_sports_result(query)
            print("\nResult:")
            print(result)
    
    text_input = widgets.Text(
        value='What was the result of yesterday\'s game between Arsenal and PSG?',
        placeholder='Enter your sports query',
        description='Query:',
        layout=widgets.Layout(width='80%')
    )
    
    button = widgets.Button(description="Search")
    output = widgets.Output()
    
    button.on_click(handle_submit)
    
    print("Interactive Sports Results Query")
    display(text_input)
    display(button)
    display(output)
    
except ImportError:
    print("IPython widgets not available. Use the function directly.")

# Advanced version: Add functionality to query multiple sources
def get_enhanced_sports_result(query: str, sources: int = 3) -> str:
    """
    Enhanced version that queries multiple sources for more accurate results.
    
    Args:
        query: A string query about sports results
        sources: Number of different search queries to run (with variations)
        
    Returns:
        A consolidated response with information from multiple sources
    """
    # Create variations of the query to get more diverse results
    query_variations = [
        query + " recent result score",
        query + " match result highlights",
        query + " final score stats"
    ]
    
    # Use only the number of variations requested
    query_variations = query_variations[:sources]
    
    # Get results for each variation
    all_results = []
    for q in query_variations:
        results = search_tool.invoke(q)
        all_results.extend(results)
    
    # Create a more complex prompt for consolidating multiple sources
    enhanced_prompt = ChatPromptTemplate.from_template("""
    You are a sports analytics expert. Given the following query and search results from multiple sources,
    provide a comprehensive and accurate answer about the sports match.
    
    Query: {query}
    
    Search Results from Multiple Sources:
    {search_results}
    
    Analyze these results and provide a detailed response that includes:
    1. The final score
    2. Date and location of the match
    3. Competition/tournament
    4. Key highlights or important moments
    5. Any notable player performances
    
    If there are conflicting results, mention this and provide the most likely correct information
    based on source reliability. If the match hasn't happened yet, provide information about when it's scheduled.
    
    Response:
    """)
    
    # Create the enhanced chain
    enhanced_chain = enhanced_prompt | llm | StrOutputParser()
    
    # Run the enhanced chain
    result = enhanced_chain.invoke({"query": query, "search_results": all_results})
    
    return result